In [10]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

17473536/17464789 [==============================] - 1s 0us/step


In [11]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          2834688   
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [21]:
# training
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 31s 48ms/step - loss: 0.4123 - acc: 0.8141 - val_loss: 0.3075 - val_acc: 0.8798
Epoch 2/10
625/625 [==============================] - 30s 49ms/step - loss: 0.2346 - acc: 0.9110 - val_loss: 0.2963 - val_acc: 0.8764
Epoch 3/10
625/625 [==============================] - 31s 50ms/step - loss: 0.1812 - acc: 0.9348 - val_loss: 0.3354 - val_acc: 0.8780
Epoch 4/10
625/625 [==============================] - 30s 49ms/step - loss: 0.1527 - acc: 0.9444 - val_loss: 0.3311 - val_acc: 0.8666
Epoch 5/10
625/625 [==============================] - 30s 49ms/step - loss: 0.1293 - acc: 0.9554 - val_loss: 0.2816 - val_acc: 0.8904
Epoch 6/10
625/625 [==============================] - 31s 49ms/step - loss: 0.1049 - acc: 0.9636 - val_loss: 0.3036 - val_acc: 0.8838
Epoch 7/10
625/625 [==============================] - 30s 48ms/step - loss: 0.0916 - acc: 0.9694 - val_loss: 0.3636 - val_acc: 0.8896
Epoch 8/10
625/625 [==============================] - 30s 48ms

In [22]:
results = model.evaluate(test_data, test_labels)
print(results)
print(test_labels)

782/782 [==============================] - 12s 15ms/step - loss: 0.4392 - acc: 0.8510
[0.43919122219085693, 0.8510400056838989]
[0 1 1 ... 0 0 0]


In [16]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

1654784/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [17]:
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [18]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


[0.9487947]
[0.3319431]
